In [ ]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset, Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd

file_path = 'C:/Users/20301162/Downloads/Dataset-Sentiment Analysis (Task-ML Engineer).xlsx'
df = pd.read_excel(file_path)
print(df.head())


                                   conversation_text sentiment
0  company_x এজেন্ট থেকে রং নাম্বারে টাকা গেলে ফে...   neutral
1  company_x গ্রাহকদের জন্য খুবই দুঃখ জনক একটা খব...  negative
2  company_x গ্রাহকদের জন্য খুবই দুঃখ জনক একটা খব...  negative
3  bank_y ব্যাংক যখন ঘোষনা দিয়েছে যে কার্ড দিয়ে ক...  negative
4  bank_y ব্যাংক যখন ঘোষনা দিয়েছে যে কার্ড দিয়ে ক...  negative


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:


# Shuffle the DataFrame
df = df.sample(frac=1, random_state=85).reset_index(drop=True)

# Split the DataFrame
train_size = 0.8
eval_size = 0.1

# Calculate sizes
train_end = int(train_size * len(df))
eval_end = train_end + int(eval_size * len(df))

# Split the data
X_train = df[:train_end].copy()
X_eval = df[train_end:eval_end].copy()
X_test = df[eval_end:].copy()

# Define the prompt generation functions
def generate_prompt(data_point):
    return f"""
            Classify the text into neutral, positive, negative and return the answer as the corresponding text classification label.
text: {data_point["conversation_text"]}
label: {data_point["sentiment"]}""".strip()

def generate_test_prompt(data_point):
    return f"""
            Classify the text into neutral, positive, negative and return the answer as the corresponding text classification label.
text: {data_point["conversation_text"]}
label: """.strip()

# Generate prompts for training and evaluation data
X_train['text'] = X_train.apply(generate_prompt, axis=1)
X_eval['text'] = X_eval.apply(generate_prompt, axis=1)

# Generate test prompts and extract true labels
y_true = X_test['sentiment']
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["text"])

In [ ]:
X_train.sentiment.value_counts()

sentiment
neutral     37
negative    33
positive     9
Name: count, dtype: int64

In [ ]:
# Convert to datasets
train_data = Dataset.from_pandas(X_train[["text"]])
eval_data = Dataset.from_pandas(X_eval[["text"]])

In [ ]:
base_model_name = "meta-llama/Meta-Llama-3.1-8B"


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype="float16",
    quantization_config=bnb_config,
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

tokenizer.pad_token_id = tokenizer.eos_token_id


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
def predict(test, model, tokenizer):
    y_pred = []
    categories = ["negative", "neutral", "positive"]

    for i in tqdm(range(len(test))):
        prompt = test.iloc[i]["text"]
        pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens=2,
                        temperature=0.1)

        result = pipe(prompt)
        answer = result[0]['generated_text'].split("label:")[-1].strip()

        # Determine the predicted category
        for category in categories:
            if category.lower() in answer.lower():
                y_pred.append(category)
                break
        else:
            y_pred.append("none")

    return y_pred

y_pred = predict(X_test, model, tokenizer)

  0%|                                                                                           | 0/11 [00:00<?, ?it/s]C:\Users\20301162\AppData\Local\anaconda3\envs\tanzir\Lib\site-packages\transformers\models\llama\modeling_llama.py:660: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:01<00:00,  6.04it/s]


In [ ]:
def evaluate(y_true, y_pred):
    labels =  ["negative", "neutral", "positive"]
    mapping = {label: idx for idx, label in enumerate(labels)}

    def map_func(x):
        return mapping.get(x, -1)  # Map to -1 if not found, but should not occur with correct data

    y_true_mapped = np.vectorize(map_func)(y_true)
    y_pred_mapped = np.vectorize(map_func)(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true_mapped, y_pred=y_pred_mapped)
    print(f'Accuracy: {accuracy:.3f}')

    # Generate accuracy report
    unique_labels = set(y_true_mapped)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true_mapped)) if y_true_mapped[i] == label]
        label_y_true = [y_true_mapped[i] for i in label_indices]
        label_y_pred = [y_pred_mapped[i] for i in label_indices]
        label_accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {labels[label]}: {label_accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true_mapped, y_pred=y_pred_mapped, target_names=labels, labels=list(range(len(labels))))
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true_mapped, y_pred=y_pred_mapped, labels=list(range(len(labels))))
    print('\nConfusion Matrix:')
    print(conf_matrix)

evaluate(y_true, y_pred)

Accuracy: 0.273
Accuracy for label negative: 0.000
Accuracy for label neutral: 0.600
Accuracy for label positive: 0.000

Classification Report:
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00         5
     neutral       0.38      0.60      0.46         5
    positive       0.00      0.00      0.00         1

   micro avg       0.38      0.27      0.32        11
   macro avg       0.12      0.20      0.15        11
weighted avg       0.17      0.27      0.21        11


Confusion Matrix:
[[0 4 0]
 [0 3 0]
 [0 1 0]]


C:\Users\20301162\AppData\Local\anaconda3\envs\tanzir\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\20301162\AppData\Local\anaconda3\envs\tanzir\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\20301162\AppData\Local\anaconda3\envs\tanzir\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [ ]:
import bitsandbytes as bnb

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  # needed for 16 bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)
modules = find_all_linear_names(model)
modules

['gate_proj', 'k_proj', 'down_proj', 'q_proj', 'up_proj', 'v_proj', 'o_proj']

In [ ]:
output_dir="C:/Users/20301162/Downloads/Sentiment/llama-3.1-before_preprocess_aug_finetuned"

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules,
)

training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=5,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    optim="adamw_torch",
    logging_steps=10,
    learning_rate=1e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=False,
    lr_scheduler_type="cosine",
    eval_strategy="epoch",
    eval_steps=None
)


trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=512,
    packing=False,
    dataset_kwargs={
    "add_special_tokens": False,
    "append_concat_token": False,
    }
)

C:\Users\20301162\AppData\Local\anaconda3\envs\tanzir\Lib\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length, dataset_kwargs. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
C:\Users\20301162\AppData\Local\anaconda3\envs\tanzir\Lib\site-packages\trl\trainer\sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
C:\Users\20301162\AppData\Local\anaconda3\envs\tanzir\Lib\site-packages\trl\trainer\sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
C:\Users\20301162\AppData\Local\anaconda3\envs\tanzir\Lib\site-pa

Map:   0%|          | 0/79 [00:00<?, ? examples/s]

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

C:\Users\20301162\AppData\Local\anaconda3\envs\tanzir\Lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()

C:\Users\20301162\AppData\Local\anaconda3\envs\tanzir\Lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
C:\Users\20301162\AppData\Local\anaconda3\envs\tanzir\Lib\site-packages\torch\utils\checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Epoch,Training Loss,Validation Loss
0,No log,1.169155
1,1.593100,0.934730
2,1.034400,0.860373
3,0.876700,0.835972
4,0.702200,0.834347


TrainOutput(global_step=45, training_loss=1.0151443428463407, metrics={'train_runtime': 147.3347, 'train_samples_per_second': 2.681, 'train_steps_per_second': 0.305, 'total_flos': 4582809745391616.0, 'train_loss': 1.0151443428463407, 'epoch': 4.556962025316456})

In [ ]:
# Save trained model and tokenizer
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

('C:/Users/20301162/Downloads/Sentiment/llama-3.1-before_preprocess_aug_finetuned\\tokenizer_config.json',
 'C:/Users/20301162/Downloads/Sentiment/llama-3.1-before_preprocess_aug_finetuned\\special_tokens_map.json',
 'C:/Users/20301162/Downloads/Sentiment/llama-3.1-before_preprocess_aug_finetuned\\tokenizer.json')

In [ ]:
y_pred = predict(X_test, model, tokenizer)
evaluate(y_true, y_pred)

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:02<00:00,  5.05it/s]

Accuracy: 0.909
Accuracy for label negative: 1.000
Accuracy for label neutral: 1.000
Accuracy for label positive: 0.000

Classification Report:
              precision    recall  f1-score   support

    negative       1.00      1.00      1.00         5
     neutral       0.83      1.00      0.91         5
    positive       0.00      0.00      0.00         1

    accuracy                           0.91        11
   macro avg       0.61      0.67      0.64        11
weighted avg       0.83      0.91      0.87        11


Confusion Matrix:
[[5 0 0]
 [0 5 0]
 [0 1 0]]



C:\Users\20301162\AppData\Local\anaconda3\envs\tanzir\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\20301162\AppData\Local\anaconda3\envs\tanzir\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\20301162\AppData\Local\anaconda3\envs\tanzir\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modi